<a href="https://colab.research.google.com/github/victoriacnelson/Beyond-Calories/blob/main/Beyond_Calories_Dataset_Creation_%26_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

from scipy.stats import norm as norm

from sklearn import datasets
from sklearn.model_selection import train_test_split , KFold
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from statistics import mode

Reading in the datasets below from the project GitHub Repository.

In [2]:
file_path = 'https://raw.githubusercontent.com/victoriacnelson/Beyond-Calories/main/data/by%20DHQ%20Food%20Group%20gi_values.csfii_94-96_foodcodes.csv'
gi_df = pd.read_csv(file_path)

In [3]:
file_path = 'https://raw.githubusercontent.com/victoriacnelson/Beyond-Calories/main/data/corgis.edu.githubfood.csv'
nutrient_df = pd.read_csv(file_path)

In [4]:
file_path = 'https://raw.githubusercontent.com/victoriacnelson/Beyond-Calories/main/data/FINAL%204.3.24%20Oxalate-Content-of-Foods%20with%20GI%20added.csv'
oxalate_gi_df = pd.read_csv(file_path)

In [5]:
gi_df.head()

,DHQ Food Group Code,DHQ Food Group Name,CSFII 1994-96 Food Code,Food Description in 1994-96 CSFII,GI Value,"GI Links to: Foster-Powel K, Holt, SHA, Brand-Miller JC. International table of glycemic index and glycemic load values: 2002. Am J Clin Nutri 2002;76:5-56."
0,5.1,"Milk, whole in cereal",11111000,"Milk, cow's, fluid, whole",27.0,"369;DAIRY PRODUCTS AND ALTERNATIVES;;Milk, ful..."
1,5.1,"Milk, whole in cereal",11121100,"Milk, dry, reconstituted, whole",27.0,"369;DAIRY PRODUCTS AND ALTERNATIVES;;Milk, ful..."
2,5.2,"Milk, 2% in cereal",11100000,"Milk, NFS",29.5,"369;DAIRY PRODUCTS AND ALTERNATIVES;;Milk, ful..."
3,5.2,"Milk, 2% in cereal",11112110,"Milk, cow's, fluid, 2% fat",29.5,"369;DAIRY PRODUCTS AND ALTERNATIVES;;Milk, ful..."
4,5.2,"Milk, 2% in cereal",11114330,"Milk, cow's, fluid, lactose reduced, 2% fat",29.5,"369;DAIRY PRODUCTS AND ALTERNATIVES;;Milk, ful..."


In [6]:
nutrient_df.head()

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,Data.Fiber,...,Data.Major Minerals.Phosphorus,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,Milk,"Milk, human",11000000,0,7,0,6.89,14,16.0,0.0,...,14,51,17,0.17,61,0.05,0.011,5.0,0.08,0.3
1,Milk,"Milk, NFS",11100000,0,4,0,4.87,8,17.9,0.0,...,103,157,39,0.42,59,0.56,0.060,0.1,0.03,0.2
2,Milk,"Milk, whole",11111000,0,7,0,4.67,12,17.8,0.0,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3
3,Milk,"Milk, low sodium, whole",11111100,0,7,0,4.46,14,16.0,0.0,...,86,253,3,0.38,29,0.36,0.034,0.9,0.08,0.3
4,Milk,"Milk, calcium fortified, whole",11111150,0,7,0,4.67,12,17.8,0.0,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3


In [7]:
oxalate_gi_df.head()

,Food Group,Subgroup,Food Item,Serving size,Oxalate Category,Oxalate Value,CSFII 1994-96 Food Code,Food Description in 1994-96 CSFII,GI Value,"GI Links to: Foster-Powel K, Holt, SHA, Brand-Miller JC. International table of glycemic index and glycemic load values: 2002. Am J Clin Nutri 2002;76:5-56.",Unnamed: 10,Unnamed: 11
0,Fruits,Whole Fruits,Avocados,1 fruit,Very High,19,63105010.0,"Avocado, raw",50.0,"1008;IMPUTED VALUES;Food, NFS;With carbohydrat...",NaN,NaN
1,Fruits,Whole Fruits,Dates,1 date,Very High,24,62110100.0,Date,103.0,"404;FRUIT AND FRUIT PRODUCTS;;;Dates, dried (A...",NaN,NaN
2,Fruits,Whole Fruits,Grapefruit,1/2 fruit,Very High,12,61101010.0,"Grapefruit, raw",25.0,"407;FRUIT AND FRUIT PRODUCTS;;;Grapefruit, raw...",NaN,NaN
3,Fruits,Whole Fruits,Kiwi,1 fruit,Very High,16,63126500.0,"Kiwi fruit, raw",53.0,"410;FRUIT AND FRUIT PRODUCTS;;Kiwi fruit, raw;...",NaN,NaN
4,Fruits,Whole Fruits,Orange,1 fruit,Very High,29,61119010.0,"Orange, raw",42.0,"415;FRUIT AND FRUIT PRODUCTS;;Oranges, raw;Mea...",NaN,NaN


# GI / Nutrient Merge

After analyzing the data in Excel, I found that CSFII 1994-96 Food Code and Nutrient Data Bank Number contain the same unique code for each food.  This will allow me to combine these datasets using this unique value.

In [8]:
gi_nutrient_df = pd.merge(gi_df, nutrient_df,
                           left_on='CSFII 1994-96 Food Code',
                           right_on='Nutrient Data Bank Number',
                           how='inner')

In [9]:
gi_nutrient_df['GI Value'] = pd.to_numeric(gi_nutrient_df['GI Value'], errors='coerce')

In [10]:
gi_nutrient_df.dtypes

DHQ Food Group Code                                                                                                                                               float64
DHQ Food Group Name                                                                                                                                                object
CSFII 1994-96 Food Code                                                                                                                                             int64
Food Description in 1994-96 CSFII                                                                                                                                  object
GI Value                                                                                                                                                          float64
GI Links to:   Foster-Powel K, Holt, SHA, Brand-Miller JC. International table of glycemic index and glycemic load values: 2002. Am J Clin Nutri 2002;

I'll go ahead and clean my data here so the dataset is ready to use after export.

In [11]:
gi_nutrient_df.dropna(subset=['GI Value'], inplace=True)

Some entries contained the string "null" rather than a NaN values.  The code below corrects this.

In [12]:
# replace 'null' strings with NaN
gi_nutrient_df = gi_nutrient_df.replace('null', pd.NA)

# count total Nan per column
nan_counts = gi_nutrient_df.isna().sum()


print(nan_counts)

DHQ Food Group Code                                                                                                                                               0
DHQ Food Group Name                                                                                                                                               0
CSFII 1994-96 Food Code                                                                                                                                           0
Food Description in 1994-96 CSFII                                                                                                                                 0
GI Value                                                                                                                                                          0
GI Links to:   Foster-Powel K, Holt, SHA, Brand-Miller JC. International table of glycemic index and glycemic load values: 2002. Am J Clin Nutri 2002;76:5-56.    0
Category        

In [13]:
#Store my file as a .csv
gi_nutrient_df.to_csv('final_gi_nutrient_4_3_24.csv', index=False)

In [ ]:
from google.colab import files

# Download the file
files.download('final_gi_nutrient_4_3_24.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Oxalate / GI Cleaning

The code below is the cleaning of the combined oxlate and glycemic index dataset.  This dataset was manually created as an excel file.  Similarly to the last dataset, there are string "null" values which need to be converted to NaNs and dropped.

In [8]:
# Deletes columns accidetally brought in from the .xslx file
oxalate_gi_df = oxalate_gi_df.drop('Unnamed: 10', axis = 1)
oxalate_gi_df = oxalate_gi_df.drop('Unnamed: 11', axis = 1)

In [15]:
oxalate_gi_df['GI Value'] = pd.to_numeric(oxalate_gi_df['GI Value'], errors='coerce')

In [ ]:
oxalate_gi_df.dropna(subset=['GI Value'], inplace=True)

In [ ]:
oxalate_gi_df = oxalate_gi_df.replace('null', pd.NA)
nan_counts = oxalate_gi_df.isna().sum()
print(nan_counts)

Food Group                                                                                                                                                        0
Subgroup                                                                                                                                                          0
Food Item                                                                                                                                                         0
Serving size                                                                                                                                                      0
Oxalate Category                                                                                                                                                  0
Oxalate Value                                                                                                                                                     0
CSFII 1994-96 Fo

In [ ]:
oxalate_gi_df['GI Value'].unique()

array([ 50. , 103. ,  25. ,  53. ,  42. ,  61. ,  38. ,  52. ,  40. ,
        22. ,  59. ,  64. ,  65. ,  46. ,  39. ,  72. ,  43.5,  58. ,
        55. ,  29. ,  31. ,  32. ,  97. ,  28. ,  37. ,  16. ,  47. ,
        42.6,  53.5,  48. ,  75. ,  72.5,  79.3,  85. ,  54. ,  65.8,
        68.2,  65.9,  27. ,  55.4,  36. ,  33. ,  19. ,  37.5,  24. ,
        29.5,  67. ,  71. , 102. ,  61.1,  66. ,  92. ,  60. ,  77. ,
        43. ,  68. ,  70. ,  73. ,  83. ,  75.5,  45. ,  69. ,  89. ,
        29.2,  95. ,   0. ,  14. ,  18. ,  20. ,  51. ,  57.9,  49. ,
        44. ,  78. ,  82. ,  62.5,  63. ,  74. ,  81. ])

In [ ]:
oxalate_gi_df.dtypes

Food Group                                                                                                                                                         object
Subgroup                                                                                                                                                           object
Food Item                                                                                                                                                          object
Serving size                                                                                                                                                       object
Oxalate Category                                                                                                                                                   object
Oxalate Value                                                                                                                                         

In [ ]:
oxalate_gi_df.columns

Index(['Food Group', 'Subgroup', 'Food Item', 'Serving size',
       'Oxalate Category', 'Oxalate Value', 'CSFII 1994-96 Food Code',
       'Food Description in 1994-96 CSFII', 'GI Value',
       'GI Links to:   Foster-Powel K, Holt, SHA, Brand-Miller JC. International table of glycemic index and glycemic load values: 2002. Am J Clin Nutri 2002;76:5-56.'],
      dtype='object')

In [ ]:
oxalate_gi_df.to_csv('final_oxalate_gi_4_3_24.csv', index=False)

In [ ]:
from google.colab import files

# Download
files.download('final_oxalate_gi_4_3_24.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Triple Threat Merge

Here I merge the three datasets together and save the .csv file.  This is achieved using an inner join on the CSFII 1994-96 Food Code.

In [ ]:
gi_nutrient_df.columns

Index(['DHQ Food Group Code', 'DHQ Food Group Name', 'CSFII 1994-96 Food Code',
       'Category', 'Description', 'Nutrient Data Bank Number',
       'Data.Alpha Carotene', 'Data.Beta Carotene', 'Data.Beta Cryptoxanthin',
       'Data.Carbohydrate', 'Data.Cholesterol', 'Data.Choline', 'Data.Fiber',
       'Data.Lutein and Zeaxanthin', 'Data.Lycopene', 'Data.Niacin',
       'Data.Protein', 'Data.Retinol', 'Data.Riboflavin', 'Data.Selenium',
       'Data.Sugar Total', 'Data.Thiamin', 'Data.Water',
       'Data.Fat.Monosaturated Fat', 'Data.Fat.Polysaturated Fat',
       'Data.Fat.Saturated Fat', 'Data.Fat.Total Lipid',
       'Data.Major Minerals.Calcium', 'Data.Major Minerals.Copper',
       'Data.Major Minerals.Iron', 'Data.Major Minerals.Magnesium',
       'Data.Major Minerals.Phosphorus', 'Data.Major Minerals.Potassium',
       'Data.Major Minerals.Sodium', 'Data.Major Minerals.Zinc',
       'Data.Vitamins.Vitamin A - RAE', 'Data.Vitamins.Vitamin B12',
       'Data.Vitamins.Vitamin B

In [ ]:
oxalate_gi_df.columns

Index(['Food Group', 'Subgroup', 'Food Item', 'Serving size',
       'Oxalate Category', 'Oxalate Value', 'CSFII 1994-96 Food Code',
       'Food Description in 1994-96 CSFII', 'GI Value',
       'GI Links to:   Foster-Powel K, Holt, SHA, Brand-Miller JC. International table of glycemic index and glycemic load values: 2002. Am J Clin Nutri 2002;76:5-56.'],
      dtype='object')

In [ ]:
gi_nutrient_df = gi_nutrient_df.drop(columns=[
    'Food Description in 1994-96 CSFII',
    'GI Value',
    'GI Links to:   Foster-Powel K, Holt, SHA, Brand-Miller JC. International table of glycemic index and glycemic load values: 2002. Am J Clin Nutri 2002;76:5-56.'
])

In [ ]:
triple_threat = pd.merge(gi_nutrient_df, oxalate_gi_df,
                         on='CSFII 1994-96 Food Code',
                         how='inner')

In [ ]:
len(triple_threat)

264

In [ ]:
triple_threat.dtypes

DHQ Food Group Code                                                                                                                                               float64
DHQ Food Group Name                                                                                                                                                object
CSFII 1994-96 Food Code                                                                                                                                             int64
Category                                                                                                                                                           object
Description                                                                                                                                                        object
Nutrient Data Bank Number                                                                                                                             

In [ ]:
triple_threat.to_csv('triple_threat_4_3_24.csv', index=False)

In [ ]:
from google.colab import files

# Trigger the browser download
files.download('triple_threat_4_3_24.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>